### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
# from resnet.plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "PACS"
MODEL_NAME = "resnet34"

c:\Users\Fatim_Sproj\anaconda3\envs\bacp\lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-25 04:38:52,853 | INFO | Initialized experiment directories for PACS
2025-10-25 04:38:52,854 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet34\PACS\logs
2025-10-25 04:38:52,854 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet34\PACS\checkpoints
2025-10-25 04:38:52,855 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet34\PACS\plots


### Setup

In [4]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [ ]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-25 04:38:53,447 | INFO | === LODO: Leaving out domain 'art_painting' ===



=== LODO: Leaving out domain 'art_painting' ===


Evaluating: 100%|██████████| 16/16 [00:07<00:00,  2.14it/s]
2025-10-25 04:39:25,746 | INFO | [art_painting] Epoch 1/5 | Train - Loss: 0.5563, Cls: 0.5474, GRQO: 0.0089, Acc: 0.8139 | Val - Loss: 0.7795, Cls: 0.7782, GRQO: 0.0013, Acc: 0.7713
2025-10-25 04:39:25,894 | INFO | [art_painting] New best val acc: 0.7713


[art_painting] Epoch 1/5 | Train - Loss: 0.5563, Cls: 0.5474, GRQO: 0.0089, Acc: 0.8139 | Val - Loss: 0.7795, Cls: 0.7782, GRQO: 0.0013, Acc: 0.7713
[art_painting] New best val acc: 0.7713


### Baseline

In [ ]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-10-21 07:55:33,053 | INFO | Initializing ResNet baseline: resnet34
2025-10-21 07:55:33,213 | INFO | === Baseline LODO: Leaving out domain 'art_painting' ===


Initializing ResNet baseline: resnet34

=== Baseline LODO: Leaving out domain 'art_painting' ===


2025-10-21 07:55:52,806 | INFO | [art_painting] Epoch 1/5 | Train - Loss: 0.3855, Acc: 0.8765 | Val Acc: 0.8121


[art_painting] Epoch 1/5 | Train - Loss: 0.3855, Acc: 0.8765 | Val Acc: 0.8121


2025-10-21 07:56:12,527 | INFO | [art_painting] Epoch 2/5 | Train - Loss: 0.0639, Acc: 0.9809 | Val Acc: 0.8081


...

[sketch] Epoch 3/5 | Train - Loss: 0.0109, Acc: 0.9988 | Val Acc: 0.6999


2025-10-21 08:02:53,471 | INFO | [sketch] Epoch 4/5 | Train - Loss: 0.0044, Acc: 0.9997 | Val Acc: 0.6564


[sketch] Epoch 4/5 | Train - Loss: 0.0044, Acc: 0.9997 | Val Acc: 0.6564


2025-10-21 08:03:19,272 | INFO | [sketch] Epoch 5/5 | Train - Loss: 0.0018, Acc: 1.0000 | Val Acc: 0.6824
2025-10-21 08:03:19,272 | INFO | [sketch] Best Val Acc: 0.6999
2025-10-21 08:03:19,272 | INFO | ------------------------------------------------------------
2025-10-21 08:03:19,272 | INFO | Baseline LODO (resnet34) finished | Mean Acc: 0.8182


[sketch] Epoch 5/5 | Train - Loss: 0.0018, Acc: 1.0000 | Val Acc: 0.6824
[sketch] Best Val Acc: 0.6999
------------------------------------------------------------
Baseline LODO (resnet34) finished | Mean Acc: 0.8182
